1. There are many occasions where you'd like to ut human in the loop to keep track on what an Agent is doing.

2. This is very easy to do with LangGraph

3. Let's see how this works.


In [ ]:
# 1. We are actually resuming from where we left in Persistence and Streaming
# 2. Let's start by setting the environmental variables.



import os
import openai
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.environ['OPENAI_API_KEY']
os.environ["TAVILY_API_KEY"] = ""

In [ ]:
# 3. We make the relevant imports and set up the check pointer

!pip install langgraph
!pip install langchain-openai
!pip install langchain-community
!pip install langgraph-checkpoint-sqlite

from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found

In [ ]:
# 4. We set the AgentState
# 5. We make one small modification
# 6. In the pervious example we annotated the message list with operator.add
# 7. This added messages to the existing messages array
# 8. For human in the loop interaction, we may also have to replace existing
# messages.
# 9. In order to that we write a custom function
# 10. We call it as: reduced_messages function
# 11. This looks for messages with the say id. And if it sees that you have
# a message with same id, it replaces that
# 12. Otherwise, it just appends.


from uuid import uuid4
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage

"""
In previous examples we've annotated the `messages` state key
with the default `operator.add` or `+` reducer, which always
appends new messages to the end of the existing messages array.

Now, to support replacing existing messages, we annotate the
`messages` key with a customer reducer function, which replaces
messages with the same `id`, and appends them otherwise.
"""
def reduce_messages(left: list[AnyMessage], right: list[AnyMessage]) -> list[AnyMessage]:
    # assign ids to messages that don't have them
    for message in right:
        if not message.id:
            message.id = str(uuid4())
    # merge the new messages with the existing messages
    merged = left.copy()
    for message in right:
        for i, existing in enumerate(merged):
            # replace any existing messages with the same id
            if existing.id == message.id:
                merged[i] = message
                break
        else:
            # append any new messages to the end
            merged.append(message)
    return merged

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]

**Creating the Tavily Tool**

In [ ]:
# 13. We now create a Tavily tool as before.
tool = TavilySearchResults(max_results=2)

/tmp/ipython-input-2573608900.py:2: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tool = TavilySearchResults(max_results=2)


**Human in the Loop. Human Approval before a Tool Call**

In [ ]:
# 14. We craete the same Agent as before.
# 15. We make one small modification.
# 16. While compiling the graph, in addition to passing the
# check pointer, we also pass interrupt_before the action parameter
# 17. This adds an interrupt before the action node
# 18. Action node is wheer we call the tool
# 19. The reason we do is that we are adding a manual approval requirement
# before we run any tools
# 20. This is when you want to amke sure the tools are executed correctly
# 21. That is with approval from Human
# 22. As mentioned, this interrupt happens before action node before tools
# are called.
# 23. It is also possible to ahve the interrupt before a certain tool call only
# 24. This is not covered here


class Agent:
    def __init__(self, model, tools, system="", checkpointer=None):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(
            checkpointer=checkpointer,
            interrupt_before=["action"]
        )
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        print(state)
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}


In [ ]:
# 25. We next initialize the Agent with the same System Prompt and model
# and checkpointer as before

prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-4.1")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)


**Streaming the Messages: Observe the Interrupt after AI Message**

In [ ]:
# 26. We now call it and we pass in the thread config with theread id of 1
# 27. This necessary for persistence as discussed before
# 28. This is a separate notebook, it will start from fresh
# 29. We stream the messages
# 30. You can notice below that the response stops after the AI message
# 31. This is because after AI message
# 32. This is because the AI message is saying we should call a tool
# 33. Snce we have interrupt_before parameter it stops before calling the tool

ctx = SqliteSaver.from_conn_string(":memory:")
memory = ctx.__enter__()  # manually enter context



abot = Agent(model, [tool], system=prompt, checkpointer=memory)
messages = [HumanMessage(content="Whats the weather in SF?")]
thread = {"configurable": {"thread_id": "1"}}

for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v.content if hasattr(v, "content") else v)

state = abot.graph.get_state(thread)
print("Current state:", state)
print("Next node to execute:", state.next)


{'messages': [HumanMessage(content='Whats the weather in SF?', additional_kwargs={}, response_metadata={}, id='1cf67838-527a-48ea-a308-e15812da7fba'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_WccREUv51PLdXImv90uhdmTd', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 150, 'total_tokens': 172, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_daf5fcc80a', 'id': 'chatcmpl-CBPI8qlh6Y1ftImOD3OyQjzNL6iui', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--6f36d7e6-550a-4e06-85b6-cef480f53aab-0', tool_calls=[{'name': 'tavily_search_resul

**Observe the State of the Graph as well as se what's the next Parameter**

In [ ]:
# 34. Observe the State of the Graph
#abot.graph.get_state(thread)
# 35. Move to the top cell

In [ ]:
# 36. Find out what's the next node to be executed
# abot.graph.get_state(thread).next
# 37. Moved to the top cell
# 38. We see that next node to eb executed is the action node

**Continuing After Interrupt: In order to continue**


In [ ]:
# 39 We continue by giving None as below
# 40 In LangGraph, the stream() method signature typically looks like:
# 41 stream(input: Optional[Any], thread: Thread)
# input: The initial input to the graph (e.g. a user message, tool result, etc.)

# thread: The execution context, which holds the current state and history

# So when you write:
# for event in abot.graph.stream(None, thread):

# You're saying:

# “Resume the graph from the current thread state, without injecting any new input.”

# 42. Doing that we see we get tool message by calling the tool
# 43. We also get the final AI message

for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_WccREUv51PLdXImv90uhdmTd', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'title\': \'San Francisco weather in September 2025 | Weather25.com\', \'url\': \'https://www.weather25.com/north-america/usa/california/san-francisco?page=month&month=September\', \'content\': \'| 14 Sunny 18° /14° | 15 Sunny 20° /13° | 16 Partly cloudy 19° /14° | 17 Partly cloudy 19° /13° | 18 Mist 18° /13° | 19 Sunny 19° /13° | 20 Sunny 22° /13° |\\n| 21 Sunny 22° /15° | 22 Partly cloudy 20° /14° | 23 Sunny 22° /13° | 24 Mist 22° /15° | 25 Partly cloudy 22° /14° | 26 Partly cloudy 20° /15° | 27 Cloudy 21° /14° |\\n| 28 Sunny 21° /13° | 29 Partly cloudy 21° /14° | 30 Sunny 22° /14° |  |  |  |  | [...] | Sun | Mon | Tue | Wed | Thu | Fri | Sat |\\n ---  ---  --- \\n|  | 1 Sunny 21° /14° | 2 Mist 16° /13° | 3 Fog 16° /13° | 4 Fog 16° /13° | 5 Sunny 19° /15° | 6 Sunny 19

In [ ]:
# 44. If we now get the state, we see mssages list contains full list of messages
# 45 NExt parameter is empty
state = abot.graph.get_state(thread)
print("Current state:", state)
print("Next node to execute:", state.next)

Current state: StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in SF?', additional_kwargs={}, response_metadata={}, id='1cf67838-527a-48ea-a308-e15812da7fba'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_WccREUv51PLdXImv90uhdmTd', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 150, 'total_tokens': 172, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_daf5fcc80a', 'id': 'chatcmpl-CBPI8qlh6Y1ftImOD3OyQjzNL6iui', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--6f36d7e6-550a-4e06-85b6-cef480f53aab-0', tool_

In [ ]:
# 46. We can write some code that runs this in a loop
# 47. And prompts us to continue or not
# 48. We pass in a new thread id so that we start afresh
messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)
while abot.graph.get_state(thread).next:
    print("\n", abot.graph.get_state(thread),"\n")
    _input = input("proceed?")
    if _input != "y":
        print("aborting")
        break
    for event in abot.graph.stream(None, thread):
        for v in event.values():
            print(v)

{'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='4df3c33b-c90e-4917-877f-efdebc6ef6a2'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_13erpvKznv4taFlSQX3uXAr0', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 150, 'total_tokens': 172, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_daf5fcc80a', 'id': 'chatcmpl-CBPIFcpHgs6tBKT0ST2TOOfaJpMtn', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--3b3514a1-abdf-4eda-a113-69d6ece23674-0', tool_calls=[{'name': 'tavily_search_results

**Modifying a State**

1. Before we get to the next section, one should undersstand about State Memory in LangGraph

2. As the Graph is executing, a snapshot of each state is stored in memory

3. If you use get_state witha  thraed id as an argument, you egt the current state

In [ ]:
# 44. LEts take an example of modifying a state
# 45. Lets take a new thread
# 46. LEt's ask an example: What's the weather in LA?

messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "3"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)


{'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='2b2f1895-221b-47e1-bee3-f9ff23c2a93c'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_dkA2PrjJiPiYhv90wknbJtQH', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 150, 'total_tokens': 172, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_daf5fcc80a', 'id': 'chatcmpl-CBPIo1dyRNAPbyl5D38SmZqWJSbEP', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--63decc93-e311-4101-b220-280db4768c3c-0', tool_calls=[{'name': 'tavily_search_results

In [ ]:
# 77. LEts say now you want to change the state
# 78. We want to modify the Agent Action
# 79. LEts save the current state of the Graph

current_values = abot.graph.get_state(thread)

In [ ]:
# 80. Let's see the last message in the state
# 81. It is the AI message
current_values.values['messages'][-1]


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_dkA2PrjJiPiYhv90wknbJtQH', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 150, 'total_tokens': 172, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_daf5fcc80a', 'id': 'chatcmpl-CBPIo1dyRNAPbyl5D38SmZqWJSbEP', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--63decc93-e311-4101-b220-280db4768c3c-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_dkA2PrjJiPiYhv90wknbJtQH', 'type': 'tool_call'}], usage_metadata={'input_token

In [ ]:
# 82. Lets see the list of tool calls associated with the message
current_values.values['messages'][-1].tool_calls


[{'name': 'tavily_search_results_json',
  'args': {'query': 'current weather in Los Angeles'},
  'id': 'call_dkA2PrjJiPiYhv90wknbJtQH',
  'type': 'tool_call'}]

In [ ]:
# 83. We update the tool call with thes evalues

_id = current_values.values['messages'][-1].tool_calls[0]['id']
current_values.values['messages'][-1].tool_calls = [
    {'name': 'tavily_search_results_json',
  'args': {'query': 'current weather in Louisiana'},
  'id': _id}
]

In [ ]:
# 84. Update with new values calling update state

abot.graph.update_state(thread, current_values.values)

{'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='2b2f1895-221b-47e1-bee3-f9ff23c2a93c'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_dkA2PrjJiPiYhv90wknbJtQH', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 150, 'total_tokens': 172, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_daf5fcc80a', 'id': 'chatcmpl-CBPIo1dyRNAPbyl5D38SmZqWJSbEP', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--63decc93-e311-4101-b220-280db4768c3c-0', tool_calls=[{'name': 'tavily_search_results

{'configurable': {'thread_id': '3',
  'checkpoint_ns': '',
  'checkpoint_id': '1f088241-9a27-69d1-8002-9ade0a2f697f'}}

In [ ]:
# 85. If we get the current state of the Graph, we can see the state term
# 86. It's current_weather in Lousiana

abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='2b2f1895-221b-47e1-bee3-f9ff23c2a93c'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_dkA2PrjJiPiYhv90wknbJtQH', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 150, 'total_tokens': 172, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_daf5fcc80a', 'id': 'chatcmpl-CBPIo1dyRNAPbyl5D38SmZqWJSbEP', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--63decc93-e311-4101-b220-280db4768c3c-0', tool_calls=[{'name': '

In [ ]:
# 87. We can get the current weather in Lousiana and get the results
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'call_dkA2PrjJiPiYhv90wknbJtQH', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'title\': \'Tuesday, September 2, 2025. New Orleans, LA\', \'url\': \'https://weathershogun.com/weather/usa/la/new-orleans/1865/september/2025-09-02\', \'content\': "WeatherShogun.com\\n\\n# Weather in New Orleans, Louisiana\\n\\n### Warning: Extreme UV Index Today: 11+\\n\\nOverexposure to the sun\'s harmful UV rays can cause sunburn and increase the risk of developing skin cancer, including melanoma. Protect yourself by wearing clothing that covers your skin, sunglasses, and a hat. Apply a broad-spectrum sunscreen with an SPF of 30 or higher every two hours, even on cloudy days. Minimize direct sun exposure during peak hours (10 AM to 4 PM) to reduce the risk. [...] ## New Orleans Weather Forecast Updated\\n\\n## Weather Forecast History\\n\\nLast Year\'s Weather on This 

In [ ]:
# 88. We have now shown how we can modify the state of the graph in order to
# control what the Agent does
# 89. An important thing to note is that we are actually keeping a running list
# of all these states
# 90. We can modify the state for it to become a new state
# 91. States get created as the graph is traversed
# 92. You can visit the states/nodes as its stored in memory.
# 93. That is: Time Travel
# 94. In order to do this, call get_state_history as below
# 95. Passing the thread id
# 96. Start building up the list of states over time

states = []
for state in abot.graph.get_state_history(thread):
    print(state)
    print('--')
    states.append(state)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='2b2f1895-221b-47e1-bee3-f9ff23c2a93c'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_dkA2PrjJiPiYhv90wknbJtQH', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 150, 'total_tokens': 172, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_daf5fcc80a', 'id': 'chatcmpl-CBPIo1dyRNAPbyl5D38SmZqWJSbEP', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--63decc93-e311-4101-b220-280db4768c3c-0', tool_calls=[{'name': '

In [ ]:
#97. You can modify the state